# Asset pricing

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats

pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns', 50)

## FM regression

In [2]:
data = pd.read_csv('input/FamaFrench.csv')
data.head()

,date,VWMe,SMB,HML,RF,S1V1,S1V2,S1V3,S1V4,S1V5,S2V1,S2V2,S2V3,S2V4,S2V5,S3V1,S3V2,S3V3,S3V4,S3V5,S4V1,S4V2,S4V3,S4V4,S4V5,S5V1,S5V2,S5V3,S5V4,S5V5
0,192607,2.62,-2.16,-2.92,0.22,3.61,-3.69,-0.64,-1.42,-0.64,3.42,3.26,2.42,-1.27,2.46,1.56,2.86,-1.34,3.34,-2.16,3.29,1.24,1.29,0.55,2.56,3.18,6.08,2.00,2.93,0.56
1,192608,2.56,-1.49,4.88,0.25,-1.94,-6.78,3.81,1.21,4.82,1.59,-2.91,3.02,2.14,7.47,-1.06,1.67,2.08,4.86,8.14,0.76,4.11,1.93,2.13,4.47,1.17,4.10,1.82,5.64,7.76
2,192609,0.36,-1.38,-0.01,0.23,-6.41,3.45,-5.19,3.08,0.75,-2.81,-2.32,1.15,-1.54,-2.61,-0.46,1.25,-2.37,1.01,-2.74,1.87,-0.08,-1.84,1.56,2.18,-1.37,3.66,-0.23,-0.30,-2.43
3,192610,-3.43,0.04,0.71,0.32,-8.66,-10.02,-3.80,0.04,-3.06,-3.65,-3.77,-7.02,-6.29,0.08,-4.80,-2.23,-0.05,-1.73,-3.73,0.13,-1.71,-2.33,-2.93,-5.21,-3.14,-3.13,-2.21,-4.59,-5.81
4,192611,2.44,-0.24,-0.31,0.31,3.77,12.42,2.31,-3.36,1.40,1.50,-2.77,4.03,3.46,1.62,1.78,4.64,3.27,3.54,3.40,3.54,2.28,3.85,5.00,1.78,4.31,2.61,1.47,3.55,2.56


In [3]:
# get only values
dates = data['date'].values
factors = data[['VWMe', 'SMB', 'HML']].values
riskfree = data['RF'].values
portfolios = data.iloc[:, 5:].values

# convert to matrix
factors = np.mat(factors)
riskfree = np.mat(riskfree)
portfolios = np.mat(portfolios)

T,K = factors.shape
N = portfolios.shape[1]
# inverse
riskfree = riskfree.T
# subtruct riskfree rate from every columns
excessReturns = portfolios - riskfree

# add 1s to first column
X = sm.add_constant(factors)

# run OLS
# Y is all 25 porfolio excess return, T=1026, shape is (1026,25)
# X first column is constant, VWMe, SMB, HML shape is (1026,4)
# every column of Y regress on X, therefore
# Y = X * beta, producing beta of shape (4,25)
ts_res = sm.OLS(excessReturns, X).fit()

In [18]:
excessReturns

matrix([[ 3.39, -3.91, -0.86, ...,  1.78,  2.71,  0.34],
        [-2.19, -7.03,  3.56, ...,  1.57,  5.39,  7.51],
        [-6.64,  3.22, -5.42, ..., -0.46, -0.53, -2.66],
        ...,
        [13.67, 15.05, 14.95, ..., 12.82,  9.16, 12.72],
        [-1.85, -1.16, -3.15, ...,  0.84,  0.72, -4.46],
        [-0.05,  2.02,  1.55, ...,  2.74,  3.2 ,  0.61]])

In [19]:
excessReturns[:,0]

matrix([[ 3.39],
        [-2.19],
        [-6.64],
        ...,
        [13.67],
        [-1.85],
        [-0.05]])

In [20]:
test_res0 = sm.OLS(excessReturns[:,0], X).fit()

In [21]:
test_res0.params

array([-0.83535616,  1.30989756,  1.28916295,  0.39425156])

In [24]:
ts_res.params[:,0]

array([-0.83535616,  1.30989756,  1.28916295,  0.39425156])

In [12]:
pd.DataFrame(ts_res.params)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,-0.835356,-0.391107,-0.121852,0.038833,0.091779,-0.239747,-0.019438,0.089810,0.048150,-0.010901,-0.155604,0.088897,0.111762,0.081767,-0.052555,0.090232,-0.010430,0.039223,0.014802,-0.176172,0.079394,0.053454,-0.023604,-0.197831,-1.273695
1,1.309898,1.085291,1.074662,0.963022,0.984967,1.069092,1.041550,0.959023,0.978756,1.050202,1.141644,1.013270,1.012869,0.961480,1.144679,1.066124,1.030785,1.009604,1.043655,1.228421,1.031042,0.957574,0.975277,1.054550,1.104513
2,1.289163,1.610011,1.181175,1.224859,1.345308,1.051988,0.988023,0.861899,0.817789,0.937324,0.788300,0.515121,0.412995,0.464647,0.497000,0.285666,0.243002,0.221419,0.201554,0.297373,-0.150711,-0.189343,-0.217260,-0.173163,0.007600
3,0.394252,0.331737,0.464803,0.585383,0.905162,-0.264721,0.187710,0.355321,0.556241,0.849301,-0.198041,0.072006,0.337946,0.506756,0.914274,-0.369234,0.132789,0.297967,0.585687,0.983352,-0.250803,-0.010672,0.312715,0.711547,0.852679


In [31]:
# first row of params is constant coefficient
# therefore alpha of 25 porfolio, shape is (25,), 1d-array
alpha = ts_res.params[0]
# shape is (3,25)
beta = ts_res.params[1:]

## not FM REG, CS REG
- CS reg: avg then reg
- FM reg: reg then avg 

In [36]:
excessReturns.shape

(1026, 25)

In [35]:
avgExcessReturns.shape

(1, 25)

In [33]:
# column-wise mean, shape (25)
avgExcessReturns = np.mean(excessReturns, 0)
# corss-section regression
# reshape avgExcessReturns to column vector (25,1)
# reshape beta to (25,3)
cs_res = sm.OLS(avgExcessReturns.T, beta.T).fit()
# producing risk premia of (3,), 1d-array
riskPremia = cs_res.params

In [40]:
fm_res = sm.OLS(excessReturns.T, beta.T).fit()

In [42]:
pd.DataFrame(fm_res.params)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025
0,2.902191,1.982859,-0.375685,-2.897510,2.659893,2.126554,0.131271,3.724361,-0.162132,-0.838004,3.754173,-1.461314,6.175553,1.849327,4.372585,-2.253312,6.820539,2.689874,-0.613849,-2.379983,9.864428,2.759236,0.774133,-4.802539,0.242596,...,3.638713,-3.451801,3.706138,6.381665,1.434721,-6.754378,-4.951884,6.828315,-4.008476,10.007383,3.568753,1.492150,6.331637,2.474398,3.773448,1.253090,3.290954,-0.803361,-1.580124,-3.054333,-6.065619,-8.106962,12.775805,0.002666,-0.059588
1,-1.733590,-3.085720,-0.612406,-1.326576,0.040849,0.111386,-1.786543,0.648455,-1.273525,1.757246,2.587442,2.313589,2.499903,0.180572,-3.554234,0.057927,2.714133,-2.030487,3.592304,-0.016965,0.153157,3.881477,5.684518,-3.886247,-0.800151,...,4.319951,0.181803,0.009290,0.917463,3.977628,0.728991,-0.715964,-0.874400,-2.703675,2.954092,1.046076,2.750212,0.556224,-2.453806,1.310708,2.084922,-0.594494,-0.365005,-0.234231,-0.295486,-2.961944,-2.281100,1.803807,-0.527685,-0.066462
2,-2.272799,5.113520,-0.184279,-0.406083,-0.648074,0.172236,4.260242,4.229522,-1.084203,2.549532,3.540680,-2.873203,-1.098447,-3.322965,0.868620,-7.266533,-0.310948,2.333917,-1.312081,-1.100756,-2.120518,1.602814,-2.513432,-0.947060,0.070132,...,0.182974,1.116048,2.230679,1.305584,2.957860,-3.045393,-4.807109,0.584958,-1.649656,-3.168021,-1.678490,-1.721987,2.801298,-0.253351,-0.107456,-1.319883,-2.035969,-1.320858,-0.197748,-0.118441,-0.808509,-0.542449,-2.256583,-0.080628,2.131815


In [43]:
pd.DataFrame(fm_res.resid)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025
0,2.719363,-2.825359,-5.285747,-3.314288,0.178655,10.372933,0.571587,-5.201992,8.261606,3.297167,-0.869135,-4.385658,12.740940,10.804868,-7.418106,-3.268227,-5.620575,-6.035981,-11.799696,3.893379,27.477183,-22.340086,10.048619,-7.905781,-1.653900,...,-4.817600,-0.302872,-2.286086,-2.716815,-1.883289,1.358406,3.644665,-2.007769,0.586550,-3.757955,1.208485,-2.781140,0.324724,-3.177968,-4.370182,1.081141,0.468271,2.043622,-2.180276,-0.711513,-0.217476,2.113861,-4.500736,-1.141433,-0.726738
1,-3.514649,-5.910276,4.674841,-4.924845,9.372465,-5.124399,-6.739391,0.250878,0.336020,1.064519,3.325247,5.414192,17.927253,-3.785431,3.998679,-10.727188,-4.918900,-8.194433,-0.162180,-4.114553,-2.208902,5.714492,-1.718501,1.973227,-0.548300,...,-2.114931,-1.606732,-0.707192,-1.396197,-2.542347,1.877048,0.911638,-1.016964,1.260561,-1.716091,0.009489,-1.896044,-0.236487,0.209261,0.140105,0.441138,-0.439094,0.787720,-0.582392,1.139865,-0.300065,-0.919042,-0.971020,-0.286568,1.484473
2,-0.874797,2.697094,-4.207253,0.749510,-0.605510,-3.776949,-2.461024,3.755738,-3.917567,1.589927,-8.556406,-6.176863,-6.158893,1.343834,-1.144616,2.600632,3.508890,3.762844,2.446400,6.649350,-10.296209,10.095058,-4.418092,-2.558361,4.151812,...,-2.138051,-0.503964,-0.040644,-1.288656,0.005064,0.763115,0.911637,0.722799,-0.901965,1.288652,1.089365,1.088345,0.756581,-0.223009,-1.823410,-1.055821,-1.628138,-0.251585,0.076680,0.286447,0.592861,1.618754,0.138582,-2.492101,0.701666
3,-0.981013,-0.163331,4.069778,4.372948,-5.902199,2.654825,4.147975,-2.066797,-3.439301,1.632187,5.952744,2.115380,-4.976211,-1.796913,-0.165936,3.342740,2.319263,2.390417,-0.530845,5.877119,-12.315944,-4.329727,6.543079,0.539448,2.035393,...,-0.642601,-0.391837,-0.616273,0.026286,1.744819,-1.805574,1.079727,-0.227226,-0.892448,-0.091172,0.744476,0.182425,-0.378635,-0.379027,0.933557,-1.357853,0.450730,-2.336059,0.054352,2.372651,0.162576,0.488751,0.808172,0.590970,0.390863
4,0.670901,2.139631,1.880715,1.626176,-0.998250,-0.530336,-0.202054,0.820853,-0.625643,1.803631,1.236468,-4.022431,-3.661584,1.133369,0.678446,-0.771103,1.802106,4.039621,7.919511,-1.426610,0.857230,5.489654,-0.464862,1.535785,0.094020,...,1.438701,1.305126,2.317953,1.468234,3.428370,-1.841303,-2.188154,0.701190,-0.561301,-0.583536,-0.073094,0.469074,-0.520376,0.183248,0.657237,-1.034408,0.141177,0.137918,-0.189523,1.583145,0.810994,1.254878,-3.177852,0.210254,-0.031534
5,1.319351,3.819939,-2.042897,0.415747,-1.868201,2.004937,2.676856,-3.104208,-0.853944,-2.137784,0.361779,3.737813,-4.112883,-3.916718,-0.825740,1.404456,5.120668,0.238163,-1.300136,-3.889126,-5.408443,-6.608847,-4.573023,3.041934,0.240958,...,1.533782,1.444478,-0.551467,-0.632130,-1.105259,0.027981,-0.925338,0.484615,1.212964,0.154879,-1.300115,-1.314282,0.677320,0.398949,0.538543,-1.462382,1.218105,-0.766811,0.843357,0.124855,0.436603,0.903179,1.036544,0.350924,0.277958
6,2.156676,-3.136343,-1.519043,0.314814,-5.769122,1.872706,-4.611269,2.096281,0.320655,-6.311947,-4.461230,3.225481,-5.065919,2.599176,5.254349,2.123703,5.332811,2.496430,1.986367,0.242255,0.972424,15.390266,-2.640935,-0.920443,-2.985273,...,1.387543,0.146105,-1.368027,0.671632,-0.879539,0.756413,1.877364,2.282093,-1.764023,2.542769,-0.285512,0.491812,-0.010110,-0.665237,1.334927,0.272648,0.171851,-0.174688,0.354323,-0.514581,-0.074122,-0.160592,-0.005260,-0.866278,0.657567
7,1.718485,1.711031,1.873601,-3.273557,1.364167,0.333383,-1.749825,1.926511,1.428378,-3.696805,-2.938530,3.988263,-0.746861,-6.578464,0.291342,2.683003,-1.019877,-0.748866,-0.971298,-0.351798,-2.548757,-3.141126,1.621183,-0.548191,-0.997925,...,1.982014,1.277106,-0.694886,0.915174,-0.755234,-0.898619,-0.825874,-0.632716,-0.489326,2.052224,-0.617726,-0.359554,0.373044,1.351949,2.199660,0.340247,-0.660285,0.634368,0.667523,-2.11406

In [38]:
cs_res.resid.shape

(25,)

## GMM